In [1]:
import os
import pandas as pd

In [2]:
SEQ_LEN = 60
FUTURE_PERIOD_PREDICT = 3
RATIO_TO_PREDICT = "LTC-USD"

In [3]:
# 1 means: SHOULD BUY
def classify(current, future):
    if float(future) > float(current):
        return 1
    else:
        return 0

In [4]:
#create empty DF
main_df = pd.DataFrame()

In [5]:
ratios = ["BTC-USD", "LTC-USD", "ETH-USD", "BCH-USD"]

for ratio in ratios:
    #for each ratio read correspondent CSV file and initialize a readable DataFrame
    dataset = f"crypto_data/{ratio}.csv"
    df = pd.read_csv(dataset, names = ["time", "low", "high", "open", "close", "volume"])

    #Renaming columns
    df.rename(columns={"close": f"{ratio}_close", "volume":f"{ratio}_volume"}, inplace = True)

    #Set correct index and clean Data (for each ratio). We are only using Volume and Close price
    df.set_index("time", inplace = True)
    df = df [[f"{ratio}_close", f"{ratio}_volume"]]

    #And join all the ratios into the same super table
    if len(main_df) == 0:
    	main_df = df
    else:
    	main_df = main_df.join(df)

In [10]:
#create future column and fill it with future values based on FUTURE_PERIOD_PREDICT variable
main_df['future'] = main_df[f"{RATIO_TO_PREDICT}_close"].shift(-FUTURE_PERIOD_PREDICT)

print(main_df[[f"{RATIO_TO_PREDICT}_close","future"]].head())

            LTC-USD_close     future
time                                
1528968660      96.580002  96.500000
1528968720      96.660004  96.389999
1528968780      96.570000  96.519997
1528968840      96.500000  96.440002
1528968900      96.389999  96.470001


In [9]:
main_df['target'] = list(map(classify, main_df[f"{RATIO_TO_PREDICT}_close"], main_df["future"]))

print(main_df[[f"{RATIO_TO_PREDICT}_close","future","target"]].head())

            LTC-USD_close     future  target
time                                        
1528968660      96.580002  96.500000       0
1528968720      96.660004  96.389999       0
1528968780      96.570000  96.519997       0
1528968840      96.500000  96.440002       0
1528968900      96.389999  96.470001       1
